In [2]:

import numpy
from scipy import integrate as I
import math 

In [2]:
def read_pars_and_matrix(file_name):
    # A quick function to read values from a .txt files and convert them to an array ora a matrix
    values = []
    
    # File opening and reading
    with open(os.path.join(data_dir, file_name),"r") as file:
        # Reading of the lines
        for line in file:
            # removal of empty spaces from the line and creation of an array of separate values
            line_values = [float(value) for value in line.strip().split()]
            #filling of original array
            values.append(line_values)
    
    # A list of values has been created, now to convert to a true array
    values_array = numpy.array(values)
    # Return of the function
    return values_array




def bolometric_fluence(data_values):
    
    a = data_values[0] #first photon index of the Band function
    b = data_values[1] #second photon index of the Band function
    g = data_values[2] #photon index of the power law
    E_p = data_values[3] #peak energy of the photon distribution
    K_b = data_values[4] #multiplicative constant of the Band function
    K_pl = data_values[5] #multiplicative constant of the power law
    z = 2.11 #redshift
    #E_max_n = 455.5, 468.5        #maximum energy binned in the n files
    E_min_b = 351.9/(1+z)        #minimum energy binned in the b files
    E_i = 1/(1+z) #keV, initial energy rescaled by the redshift
    Eb_c = (a - b)*E_p/(1+z) #keV, characteristic energy of the Band function, divides the two branches, this too needs to be rescaled by the redshift
    E_f = 10000/(1+z) #keV, final energy, set by default to 10 MeV, rescaled by the redshift

    def inferior_band_plus_pl(E):
        return E*(K_b*((E/100)**a)*math.exp(-(E/E_p)) )
    I_inf, I_inf_err = I.quad(inferior_band_plus_pl, E_i, Eb_c)

    
    def superior_band_plus_pl(E):
        return E*(((E/100)**a)*(((E_p/100)*(a-b))**(a-b))*math.exp(b-a) )
    I_sup, I_sup_err = I.quad(superior_band_plus_pl, Eb_c, E_f)
    
    
    def powerlaw(E):
        return K_pl*E**(-g)
    I_pl, I_pl_err = I.quad(powerlaw, E_min_b, E_f) 
    

    Bol_flu = I_inf + I_sup + I_pl #keV/cm^2, integrated bolometric fluence
    Bol_flu_erg = Bol_flu*(1.6*10**(-9))
    
    D_l = 5.10*10**28 #cm, luminosity distance of the grb

    E_iso = ((4*numpy.pi)*(D_l**2)*Bol_flu_erg)/(1+z) 
    E_iso_rescaled = E_iso/(10**52)
    
    return(E_iso_rescaled)


def bolometric_fluence_error(data, covariance_matrix):
    mean_values = data.flatten() #initialization of the mean values of the parameters
    cov_matrix = covariance_matrix #initialization of the covariance matrix
    list_of_S = []
    
    for i in range(10):
        sample = numpy.random.multivariate_normal(mean_values, cov_matrix)
        S_value = bolometric_fluence(sample)
        list_of_S.append(S_value)
        
    array_of_S = numpy.array(list_of_S)
    Bol_error = numpy.std(array_of_S)
    return(Bol_error)

def bolometric_fluence_error_alt(data, covariance_matrix):
    mean_values = data.flatten() #initialization of the mean values of the parameters
    cov_matrix = covariance_matrix #initialization of the covariance matrix
    list_of_S = []
    

    samples = numpy.random.multivariate_normal(mean_values, cov_matrix, 10)
    S_values = [bolometric_fluence(pars) for pars in samples] 
    Bol_error = numpy.std(S_values)
    return(Bol_error)

In [3]:
import os 
import sys


sys.path.append("/home/jovyan/experience-analysis-of-grb-emission-mmlab02")
main_dir =os.getcwd()
data_dir = os.path.join(main_dir,"data/current")
results_dir = os.path.join(main_dir, 'results')
print(main_dir)
print(data_dir)
print(results_dir)    
    

/home/jovyan/experience-analysis-of-grb-emission-mmlab02
/home/jovyan/experience-analysis-of-grb-emission-mmlab02/data/current
/home/jovyan/experience-analysis-of-grb-emission-mmlab02/results


In [4]:

# name of the file containing the data
parameters_filename = 'test_pars.txt'

# Call to function to read values in file and make an array or matrix
data_values = read_pars_and_matrix(parameters_filename)
# Control print
print("Array ottenuto:")
print(data_values)

E_iso = bolometric_fluence(data_values)
z = 2.11

print(E_iso)

Array ottenuto:
[[-8.600e-01]
 [-5.020e+00]
 [ 7.100e-01]
 [ 2.531e+02]
 [ 7.000e-02]
 [ 3.000e-03]]
198.9775329079999


In [5]:
matrix_filename = 'test_matrix.txt'
covariance_matrix = read_pars_and_matrix(matrix_filename)
print(covariance_matrix)

S_error = bolometric_fluence_error_alt(data_values, covariance_matrix)
print(S_error)

[[1.  0.5 0.3 0.2 0.1 0.4]
 [0.5 1.  0.6 0.4 0.2 0.7]
 [0.3 0.6 1.  0.7 0.4 0.8]
 [0.2 0.4 0.7 1.  0.5 0.9]
 [0.1 0.2 0.4 0.5 1.  0.6]
 [0.4 0.7 0.8 0.9 0.6 1. ]]
16930.74934016904


In [6]:
data = read_pars_and_matrix(parameters_filename)
mean_values = data.flatten()
matrix_filename = 'test_matrix.txt'
covariance_matrix = read_pars_and_matrix(matrix_filename)


for i in range(5):  # esegui il loop 10 volte
    sample = numpy.random.multivariate_normal(mean_values, covariance_matrix)
    print(sample)
    S_value = bolometric_fluence(sample)
    print(S_value)

[-9.29428990e-01 -4.19948970e+00  1.64468995e+00  2.54463824e+02
 -9.51106725e-03  1.11910470e+00]
23.288680872222056
[-1.64244559e+00 -5.27394019e+00  4.47899381e-01  2.52501498e+02
  1.22247726e+00 -2.21848499e-01]
7.787350346369882
[-2.00060664e-01 -4.95704106e+00  2.26391985e+00  2.53933260e+02
  4.55158060e-01  8.43902059e-01]
5688.220965732514
[-1.26837692e+00 -6.19572862e+00  1.18978681e-01  2.51548630e+02
  1.60741817e+00 -9.77294888e-01]
402.506539523041
[-1.26084842e+00 -6.15562968e+00  2.26862681e-01  2.52171104e+02
  6.25294941e-01 -8.50409916e-01]
383.32611470674624


In [20]:
#Simple test of integral calculation

"""Definition of the characterizing constants"""
a = -0.81 #first photon index of the Band function
b = -9.37 #second photon index of the Band function
g = 1.1 #photon index of the power law
E_p = 280 #peak energy of the photon distribution
K_b = 0.075 #multiplicative constant of the Band function
K_pl = 2 #multiplicative constant of the power law
z = 2.11 #redshift
"""Definition of the extremes of integration"""
E_i = 1/(1+z) #keV, initial energy rescaled by the redshift
Eb_c = (a - b)*E_p/(1+z) #keV, characteristic energy of the Band function, divides the two branches, this too needs to be rescaled by the redshift
print("Eb_c:", Eb_c)
E_c = 2000/(1+z)
E_f = 10000/(1+z) #keV, final energy, set by default to 10 MeV, rescaled by the redshift

"""Definition of the two branches of the function"""
#inferior branch, to be integrated from E_i to Eb_c
def inferior_band(E):

    return E*(K_b*((E/100)**a)*math.exp(-(E/E_p)))


#superior branch, to be integrated from Eb_c to E_f
def superior_band(E):
    
    return E*(K_b*((E/100)**b)*(((E_p/100)*(a-b))**(a-b))*math.exp(b-a) + (K_pl*E**(-g)))

def powerlaw(E):

    return E*(K_pl*E**(-g))

I_inf, I_inf_err = I.quad(inferior_band, E_i, Eb_c)
print("I_inf", I_inf)
I_sup, I_sup_err = I.quad(superior_band, Eb_c, E_f)
print("I_sup", I_sup)
I_pl, I_pl_err = I.quad(powerlaw, E_c, E_f)
print("I_pl", I_pl)
S = I_inf + I_sup #keV/cm^2, integrated bolometric fluence
print(S)
S_erg = S*(1.6*10**(-9))
errore_assoluto = numpy.sqrt(((I_inf_err)**2) + ((I_sup_err)**2))

print("Il risultato dell'integrale è:", S_erg)
print("L'errore assoluto stimato è:", errore_assoluto)

"""Calculation of the isotropic energy of the grb"""
D_l = 5.10*10**28 #cm, luminosity distance of the grb

E_iso = ((4*numpy.pi)*(D_l**2)*S_erg)/(1+z) 
E_iso_rescaled = E_iso/(10**52)

print("L'energia isotropica è %f 10^52 erg"  %(E_iso_rescaled))


Eb_c: 770.6752411575562
I_inf 2142.5955693621368
I_sup 5964.975891423956
I_pl 2437.854797013864
8107.571460786094
Il risultato dell'integrale è: 1.2972114337257753e-05
L'errore assoluto stimato è: 1.441308289765173e-07
L'energia isotropica è 13.633288 10^52 erg


In [21]:
a = -0.81 + 0.03 #first photon index of the Band function
b = -9.37 #second photon index of the Band function
g = 1.1 #photon index of the power law
E_p = 280 + 27 #peak energy of the photon distribution
K_b = 0.08 #multiplicative constant of the Band function
K_pl = 2 #multiplicative constant of the power law
z = 2.11 #redshift
"""Definition of the extremes of integration"""
E_i = 1/(1+z) #keV, initial energy rescaled by the redshift
Eb_c = (a - b)*E_p/(1+z) #keV, characteristic energy of the Band function, divides the two branches, this too needs to be rescaled by the redshift
print("Eb_c:", Eb_c)
E_c = 2000/(1+z)
E_f = 10000/(1+z) #keV, final energy, set by default to 10 MeV, rescaled by the redshift

"""Definition of the two branches of the function"""
#inferior branch, to be integrated from E_i to Eb_c
def inferior_band(E):

    return E*(K_b*((E/100)**a)*math.exp(-(E/E_p)))


#superior branch, to be integrated from Eb_c to E_f
def superior_band(E):
    
    return E*(K_b*((E/100)**b)*(((E_p/100)*(a-b))**(a-b))*math.exp(b-a) + (K_pl*E**(-g)))

def powerlaw(E):

    return E*(K_pl*E**(-g))

I_inf, I_inf_err = I.quad(inferior_band, E_i, Eb_c)
print("I_inf", I_inf)
I_sup, I_sup_err = I.quad(superior_band, Eb_c, E_f)
print("I_sup", I_sup)
I_pl, I_pl_err = I.quad(powerlaw, E_c, E_f)
print("I_pl", I_pl)
S = I_inf + I_sup #keV/cm^2, integrated bolometric fluence
print(S)
S_erg = S*(1.6*10**(-9))
errore_assoluto = numpy.sqrt(((I_inf_err)**2) + ((I_sup_err)**2))

print("Il risultato dell'integrale è:", S_erg)
print("L'errore assoluto stimato è:", errore_assoluto)

"""Calculation of the isotropic energy of the grb"""
D_l = 5.10*10**28 #cm, luminosity distance of the grb

E_iso = ((4*numpy.pi)*(D_l**2)*S_erg)/(1+z) 
E_iso_rescaled_sup = E_iso/(10**52)

print("L'energia isotropica è %f 10^52 erg"  %(E_iso_rescaled_sup))

Eb_c: 847.9517684887461
I_inf 2605.2543496701815
I_sup 6908.336748242467
I_pl 2437.854797013864
9513.59109791265
Il risultato dell'integrale è: 1.5221745756660241e-05
L'errore assoluto stimato è: 5.279650133103558e-05
L'energia isotropica è 15.997580 10^52 erg


In [22]:
a = -0.81 - 0.03 #first photon index of the Band function
b = -9.37 #second photon index of the Band function
g = 1.1 #photon index of the power law
E_p = 280 - 27 #peak energy of the photon distribution
K_b = 0.07 #multiplicative constant of the Band function
K_pl = 2 #multiplicative constant of the power law
z = 2.11 #redshift
"""Definition of the extremes of integration"""
E_i = 1/(1+z) #keV, initial energy rescaled by the redshift
Eb_c = (a - b)*E_p/(1+z) #keV, characteristic energy of the Band function, divides the two branches, this too needs to be rescaled by the redshift
print("Eb_c:", Eb_c)
E_c = 2000/(1+z)
E_f = 10000/(1+z) #keV, final energy, set by default to 10 MeV, rescaled by the redshift

"""Definition of the two branches of the function"""
#inferior branch, to be integrated from E_i to Eb_c
def inferior_band(E):

    return E*(K_b*((E/100)**a)*math.exp(-(E/E_p)))


#superior branch, to be integrated from Eb_c to E_f
def superior_band(E):
    
    return E*(K_b*((E/100)**b)*(((E_p/100)*(a-b))**(a-b))*math.exp(b-a) + (K_pl*E**(-g)))

def powerlaw(E):

    return E*(K_pl*E**(-g))

I_inf, I_inf_err = I.quad(inferior_band, E_i, Eb_c)
print("I_inf", I_inf)
I_sup, I_sup_err = I.quad(superior_band, Eb_c, E_f)
print("I_sup", I_sup)
I_pl, I_pl_err = I.quad(powerlaw, E_c, E_f)
print("I_pl", I_pl)
S = I_inf + I_sup #keV/cm^2, integrated bolometric fluence
print(S)
S_erg = S*(1.6*10**(-9))
errore_assoluto = numpy.sqrt(((I_inf_err)**2) + ((I_sup_err)**2))

print("Il risultato dell'integrale è:", S_erg)
print("L'errore assoluto stimato è:", errore_assoluto)

"""Calculation of the isotropic energy of the grb"""
D_l = 5.10*10**28 #cm, luminosity distance of the grb

E_iso = ((4*numpy.pi)*(D_l**2)*S_erg)/(1+z) 
E_iso_rescaled_inf = E_iso/(10**52)

print("L'energia isotropica è %f 10^52 erg"  %(E_iso_rescaled_inf))

Eb_c: 693.9196141479099
I_inf 1746.7994009138044
I_sup 5217.880580553799
I_pl 2437.854797013864
6964.679981467603
Il risultato dell'integrale è: 1.1143487970348166e-05
L'errore assoluto stimato è: 5.468831798529471e-08
L'energia isotropica è 11.711458 10^52 erg


In [23]:
sup = E_iso_rescaled_sup - E_iso_rescaled
inf = E_iso_rescaled - E_iso_rescaled_inf
print(E_iso_rescaled_sup)
print(E_iso_rescaled)
print(E_iso_rescaled_inf)
print(sup)
print(inf)


15.99758001271717
13.63328755859659
11.711458283186404
2.36429245412058
1.921829275410186


In [20]:
E_p = 288
E_p_sup = 288 + 94
E_p_inf = 288 - 94
E_p_int = (1+z)*E_p
E_p_int_sup = (1+z)*E_p_sup
E_p_int_inf = (1+z)*E_p_inf
DE = (E_p_int_sup - E_p_int_inf)/2
print("Energia di picco intrinseca:", E_p_int)
print("con errore:", DE)
a = 1188.02
DE_alt = (1+z)*94
print(DE_alt)

Energia di picco intrinseca: 895.68
con errore: 292.34
292.34
